In [ ]:
import json, os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['axes.facecolor'] = 'white'
import matplotlib.colors

import pytz 

from datetime import timedelta, datetime

from research.utils.data_access_utils import RDSAccessUtils

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
rds_access_utils = RDSAccessUtils(json.load(open(os.environ['PROD_SQL_CREDENTIALS'])))

query = """
    select p.id as id, s.name as site_name, p.name as pen_name from pens p
    left join sites s
    on p.site_id = s.id
    order by p.id;
"""

df_pens = rds_access_utils.extract_from_database(query)

pen_infos = {}

for index, pen in df_pens.iterrows():
    pen_infos[pen.id] = pen

In [ ]:
rds_access_utils = RDSAccessUtils(json.load(open(os.environ['DATA_WAREHOUSE_SQL_CREDENTIALS'])))

In [ ]:
queryCache = {}

In [ ]:
pen_ids = [ 100, 131, 60, 56, 128, 122, 114, 1, 4 ]

fig, axes = plt.subplots(len(pen_ids), 1, figsize=(10, len(pen_ids) * 3))
fig.tight_layout(pad = 3)


start_date = '2020-08-17'
end_date = '2020-08-26'

for index, pen_id in enumerate(pen_ids):
    query = """
            select captured_at, left_crop_metadata 
            from prod.crop_annotation
            where pen_id={0}
            and group_id='{0}'
            and captured_at between '{1}' and '{2}'
            and annotation_state_id in (1, 8)
            and service_id=1;
        """.format(pen_id, start_date, end_date)
    
    if query in queryCache:
        df = queryCache[query].copy()
    else:
        df = rds_access_utils.extract_from_database(query)
        queryCache[query] = df.copy()
        
    df = df.sort_values('captured_at', ascending=True)
    df.index = pd.to_datetime(df.captured_at)
    df['hour'] = df.index.hour.values
    df['date'] = df.index.date.astype(str)
    vals = df.left_crop_metadata.apply(lambda x: x.get('mean_luminance') if x else None)
    df['mean_luminance'] = vals

    byHour = df.groupby(['hour']).mean()

    axes[index].bar(byHour.index, byHour['mean_luminance'])
    axes[index].set_xlabel('Hour')
    axes[index].set_ylabel('Luminance')
    axes[index].set_title('%i: %s %s (8/17/20 - 8/26/20)' % (pen_id, pen_infos[pen_id]['site_name'], pen_infos[pen_id]['pen_name']))
